In [1]:
#carregamento de bibliotecas

from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import time
from six import StringIO
from IPython.display import Image  
import pydotplus
import import_ipynb
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import itertools
from controllers.utils.utils import show_title, mostrar_dataframe_interativo

show_title("bibliotecas carregadas")


In [2]:
#configuracoes e variaveis iniciais

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.6f}'.format
iterations_count = 10;
algoritms = ['knn', 'dt', 'svm', 'mlp']

show_title("configurações iniciais concluídas")


In [3]:
#carregamendo dos dados

df = pd.read_csv("data.csv")

show_title(f"dados carregados: {df['cid'].count()} registros")


In [4]:
#ETL

df = df.drop(columns=['pidnum']) #pidnum é a coluna de identificação do paciente, não tem significado para o treinamento

# Identifica e remove colunas com valores iguais em todas as linhas
for col in df.columns:
    if df[col].nunique() == 1:        
        df = df.drop(columns=[col])
        print(f"coluna {col} excluida")
        
show_title("ETL concluido")

coluna zprior excluida


In [5]:
# Embaralhando as linhas do DataFrame

df = df.sample(frac=1, random_state=None).reset_index(drop=True)
show_title("Dados embaralhados")
mostrar_dataframe_interativo(df)


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [6]:
# Extratificação 

X = df.drop(columns=['cid'])  # Variáveis independentes (todas as colunas menos a 'cid')
y = df['cid']  # Variável dependente (alvo)

show_title("Variaveis independentes")
mostrar_dataframe_interativo(X)



IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [7]:
show_title("Variavel dependente (target)")
mostrar_dataframe_interativo(y)

IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [8]:
# Contagem das classes antes da divisão

show_title("Distribuição original das classes em 'cid':")
print(y.value_counts(normalize=True))  # Proporção das classes antes da divisão


cid
0   0.756428
1   0.243572
Name: proportion, dtype: float64


In [9]:
# Primeiro, dividir em treino (50%) e restante (50% para validação + teste)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

show_title("dados splitados para treinamento")

In [10]:
# Contagem das classes no conjunto de treino

show_title("Distribuição das classes no conjunto de treino (y_train):")
print(y_train.value_counts(normalize=True))  # Proporção das classes no treino


cid
0   0.756782
1   0.243218
Name: proportion, dtype: float64


In [11]:
# Segunda divisão: validação (25%) e teste (25%)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

show_title("dados de validacao e teste splitados")


In [12]:
# Contagem das classes nos conjuntos de validação e teste

show_title("Distribuição das classes no conjunto de validação (y_val):")
print(y_val.value_counts(normalize=True))  # Proporção das classes na validação
print()

show_title("Distribuição das classes no conjunto de teste (y_test):")
print(y_test.value_counts(normalize=True))  # Proporção das classes no teste
print()


cid
0   0.755140
1   0.244860
Name: proportion, dtype: float64



cid
0   0.757009
1   0.242991
Name: proportion, dtype: float64



In [13]:
#funcao para rodar o classificar e armazenar os resultados

def update_results(results,algoritm,iteration,y_val, classifier):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_val)  # Usando X_val e y_val para validação
    iteration['accuracy'] = metrics.accuracy_score(y_val, y_pred)
    iteration['confusion_matrix'] = metrics.confusion_matrix(y_val, y_pred).tolist()
    iteration['elapsed_time'] = time.time() - iteration['start_time']
    
    results[algoritm]['iterations'].append(iteration)

    # Atualizar melhores e piores índices e valores de acurácia e tempo
    if not results[algoritm]['best_accuracy'] or iteration['accuracy'] > results[algoritm]['best_accuracy']['accuracy']:
        results[algoritm]['best_accuracy'] = iteration

    if not results[algoritm]['worst_accuracy'] or iteration['accuracy'] < results[algoritm]['worst_accuracy']['accuracy']:
        results[algoritm]['worst_accuracy'] = iteration
        
    if not results['best_metrics']['best_accuracy'] or results['best_metrics']['best_accuracy']['accuracy'] < iteration['accuracy']: 
        results['best_metrics']['best_accuracy'] = dict(iteration)
        results['best_metrics']['best_accuracy']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_accuracy'] or results['worst_metrics']['worst_accuracy']['accuracy'] > iteration['accuracy']: 
        results['worst_metrics']['worst_accuracy'] = dict(iteration)
        results['worst_metrics']['worst_accuracy']['algoritm'] = algoritm

    if not results[algoritm]['best_time'] or iteration['elapsed_time'] < results[algoritm]['best_time']['elapsed_time']:
        results[algoritm]['best_time'] = iteration

    if not results[algoritm]['worst_time'] or iteration['elapsed_time'] > results[algoritm]['worst_time']['elapsed_time']:
        results[algoritm]['worst_time'] = iteration
        
    if not results['best_metrics']['best_time'] or results['best_metrics']['best_time']['elapsed_time'] > iteration['elapsed_time']: 
        results['best_metrics']['best_time'] = dict(iteration)
        results['best_metrics']['best_time']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_time'] or results['worst_metrics']['worst_time']['elapsed_time'] < iteration['elapsed_time']: 
        results['worst_metrics']['worst_time'] = dict(iteration)
        results['worst_metrics']['worst_time']['algoritm'] = algoritm

show_title("função de classificacao e atualização de resultados criada")

In [14]:
# Dicionários para armazenar resultados de validação e teste

validation_results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

test_results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

for alg in algoritms:
    validation_results[alg] = {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None}
    test_results[alg] = {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None}

show_title("dicionario de resultados de validação e teste criado")

In [15]:
#LOOP DE EXECUSSÕES DE VALIDAÇÃO (LOOP 1)

for i in range(iterations_count):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':{
            'k':np.random.randint(1, 20), # Valor de K aleatório 
            'weights': np.random.choice(['uniform', 'distance']) # Pesos aleatórios
        },
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(validation_results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':{
            'max_depth': np.random.randint(1, 100),  # Aleatório entre 
            'min_samples_split': np.random.randint(2, 30),  # Aleatório entre 2 e 10 (influencia na poda)
            'criterion': np.random.choice(['gini', 'entropy']),  # Critério aleatório
            'ccp_alpha': np.random.uniform(0.0, 0.1)   # Parâmetro de complexidade de poda entre 0 e 0.05 (influencia na poda)
        },
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'], ccp_alpha=dt_iteration['parameters']['ccp_alpha'])
    update_results(validation_results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':{
            'C': np.random.uniform(0.1, 1.0),  # Valor de erro aleatório (C)
            'kernel': np.random.choice(['rbf', 'poly'])  # Tipo de Kernel aleatório (linear excluido por ser mais demorado)
        },
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(validation_results,'svm',svm_iteration,y_val,svm_clf) 

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':{
            'epochs': np.random.randint(100, 301),  # Número de épocas aleatório entre 100 e 300
            'learning_rate': np.random.uniform(0.001, 0.1),  # Taxa de aprendizagem aleatória
            'hidden_layers': np.random.randint(1, 6),  # Número de camadas escondidas aleatório entre 1 e 5
            'activation': np.random.choice(['identity', 'logistic', 'tanh', 'relu']),  # Função de ativação aleatória
            'tol': 1e-2  # Tolerância um pouco maior para convergência
        },
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'], tol=mlp_iteration['parameters']['tol'])
    update_results(validation_results,'mlp',mlp_iteration,y_val,mlp_clf) 

    validation_results['iterations'] += 1

validation_results['elapsed_time'] = time.time() - validation_results['start_time']

show_title("loop de validacao concluido")


In [16]:
# Exibir resultados de validação
show_title("mostrar json ? (descomente)")
print(json.dumps(validation_results, indent=4))

{
    "iterations": 10,
    "start_time": 1730244109.300233,
    "elapsed_time": 2.1558077335357666,
    "best_metrics": {
        "best_accuracy": {
            "iteration": 1,
            "parameters": {
                "max_depth": 22,
                "min_samples_split": 19,
                "criterion": "entropy",
                "ccp_alpha": 0.01758376713255776
            },
            "start_time": 1730244109.7320082,
            "accuracy": 0.8990654205607477,
            "confusion_matrix": [
                [
                    384,
                    20
                ],
                [
                    34,
                    97
                ]
            ],
            "elapsed_time": 0.016695261001586914,
            "algoritm": "dt"
        },
        "best_time": {
            "iteration": 5,
            "parameters": {
                "max_depth": 38,
                "min_samples_split": 28,
                "criterion": "gini",
                "ccp_alpha": 

In [17]:
#tabulação dos resultados da validação e calculo das medias

validation_accuracy_table = {
    'repetição':[]
}
validation_time_table = {
    'repetição':[]
}
validation_accuracy_avgs = {}
validation_time_avgs = {}

for alg in algoritms:
    validation_accuracy_table[alg] = []
    validation_accuracy_avgs[alg] = 0
    validation_time_table[alg] = []
    validation_time_avgs[alg] = 0

for i in range(iterations_count):
    validation_accuracy_table['repetição'].append(i+1)  
    validation_time_table['repetição'].append(i+1)  
    for alg in algoritms:
        if len(validation_results[alg]['iterations']) > 0:
            validation_accuracy_table[alg].append(validation_results[alg]['iterations'][i]['accuracy'])
            validation_accuracy_avgs[alg] += validation_results[alg]['iterations'][i]['accuracy']
            validation_time_table[alg].append(validation_results[alg]['iterations'][i]['elapsed_time'])
            validation_time_avgs[alg] += validation_results[alg]['iterations'][i]['elapsed_time']
        else:
            validation_accuracy_table[alg].append(0)
            validation_time_table[alg].append(float('inf'))

validation_accuracy_table['repetição'].append('AVG')
validation_accuracy_table['repetição'].append('WORST')
validation_accuracy_table['repetição'].append('BEST')
validation_time_table['repetição'].append('AVG')
validation_time_table['repetição'].append('WORST')
validation_time_table['repetição'].append('BEST')
for alg in algoritms:
    validation_accuracy_avgs[alg] = validation_accuracy_avgs[alg] / iterations_count
    validation_time_avgs[alg] = validation_time_avgs[alg] / iterations_count
    validation_accuracy_table[alg].append(validation_accuracy_avgs[alg])
    validation_accuracy_table[alg].append(validation_results[alg]['worst_accuracy']['accuracy'])
    validation_accuracy_table[alg].append(validation_results[alg]['best_accuracy']['accuracy'])
    validation_time_table[alg].append(validation_time_avgs[alg])
    validation_time_table[alg].append(validation_results[alg]['worst_time']['elapsed_time'])
    validation_time_table[alg].append(validation_results[alg]['best_time']['elapsed_time'])

validation_accuracy_table = pd.DataFrame(validation_accuracy_table)
validation_time_table = pd.DataFrame(validation_time_table)

print()
show_title("Medias de acuracias na validação")
print(validation_accuracy_table)
#mostrar_dataframe_interativo(validation_accuracy_table,min(len(validation_accuracy_table),iterations_count)+1,min(len(validation_accuracy_table),iterations_count)+1)
print()
show_title("Medias de tempos na validação")
print(validation_time_table)
#mostrar_dataframe_interativo(validation_time_table,min(len(validation_time_table),iterations_count)+1,min(len(validation_time_table),iterations_count)+1)

   repetição      knn       dt      svm      mlp
0          1 0.863551 0.841121 0.824299 0.755140
1          2 0.857944 0.899065 0.850467 0.755140
2          3 0.841121 0.841121 0.842991 0.755140
3          4 0.869159 0.841121 0.785047 0.788785
4          5 0.857944 0.841121 0.848598 0.755140
5          6 0.816822 0.841121 0.837383 0.755140
6          7 0.816822 0.863551 0.781308 0.755140
7          8 0.863551 0.841121 0.852336 0.755140
8          9 0.857944 0.899065 0.850467 0.755140
9         10 0.871028 0.841121 0.848598 0.788785
10       AVG 0.851589 0.854953 0.832150 0.761869
11     WORST 0.816822 0.841121 0.781308 0.755140
12      BEST 0.871028 0.899065 0.852336 0.788785



   repetição      knn       dt      svm      mlp
0          1 0.174563 0.008986 0.069015 0.029944
1          2 0.052774 0.016695 0.035115 0.044284
2          3 0.102776 0.010194 0.067486 0.042877
3          4 0.051517 0.006006 0.106621 0.057408
4          5 0.061657 0.016738 0.085332 0.025187
5          6 0.064229 0.000908 0.101038 0.045888
6          7 0.093378 0.002394 0.055929 0.024774
7          8 0.100217 0.009598 0.072013 0.042482
8          9 0.060815 0.016689 0.059683 0.042347
9         10 0.059013 0.020593 0.051214 0.069937
10       AVG 0.082094 0.010880 0.070345 0.042513
11     WORST 0.174563 0.020593 0.106621 0.069937
12      BEST 0.051517 0.000908 0.035115 0.024774


In [18]:
#LOOP DE EXECUSSÕES DE TESTE (LOOP 2)

for i in range(iterations_count):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':validation_results['knn']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(test_results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':validation_results['dt']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'], ccp_alpha=dt_iteration['parameters']['ccp_alpha'])
    update_results(test_results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':validation_results['svm']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(test_results,'svm',svm_iteration,y_val,svm_clf)

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':validation_results['mlp']['best_accuracy']['parameters'],
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'],tol=mlp_iteration['parameters']['tol'])
    update_results(test_results,'mlp',mlp_iteration,y_val,mlp_clf) 

    test_results['iterations'] += 1

test_results['elapsed_time'] = time.time() - test_results['start_time']

show_title("loop de testes concluido")

In [19]:
# Exibir resultados de teste
show_title("mostrar json ? (descomente)")
print(json.dumps(test_results, indent=4))

{
    "iterations": 10,
    "start_time": 1730244109.3012316,
    "elapsed_time": 12.630815267562866,
    "best_metrics": {
        "best_accuracy": {
            "iteration": 0,
            "parameters": {
                "max_depth": 22,
                "min_samples_split": 19,
                "criterion": "entropy",
                "ccp_alpha": 0.01758376713255776
            },
            "start_time": 1730244111.8650289,
            "accuracy": 0.8990654205607477,
            "confusion_matrix": [
                [
                    384,
                    20
                ],
                [
                    34,
                    97
                ]
            ],
            "elapsed_time": 0.008630990982055664,
            "algoritm": "dt"
        },
        "best_time": {
            "iteration": 0,
            "parameters": {
                "max_depth": 22,
                "min_samples_split": 19,
                "criterion": "entropy",
                "ccp_alph

In [20]:
#tabulação dos resultados da testes e calculo das medias

test_accuracy_table = {
    'repetição':[]
}
test_time_table = {
    'repetição':[]
}
test_accuracy_avgs = {}
test_time_avgs = {}

for alg in algoritms:
    test_accuracy_table[alg] = []
    test_accuracy_avgs[alg] = 0
    test_time_table[alg] = []
    test_time_avgs[alg] = 0

for i in range(iterations_count):
    test_accuracy_table['repetição'].append(i+1)  
    test_time_table['repetição'].append(i+1)  
    for alg in algoritms:
        if len(test_results[alg]['iterations']) > 0:
            test_accuracy_table[alg].append(test_results[alg]['iterations'][i]['accuracy'])
            test_accuracy_avgs[alg] += test_results[alg]['iterations'][i]['accuracy']
            test_time_table[alg].append(test_results[alg]['iterations'][i]['elapsed_time'])
            test_time_avgs[alg] += test_results[alg]['iterations'][i]['elapsed_time']
        else:
            test_accuracy_table[alg].append(0)
            test_time_table[alg].append(float('inf'))

test_accuracy_table['repetição'].append('AVG')
test_accuracy_table['repetição'].append('WORST')
test_accuracy_table['repetição'].append('BEST')
test_time_table['repetição'].append('AVG')
test_time_table['repetição'].append('WORST')
test_time_table['repetição'].append('BEST')
for alg in algoritms:
    test_accuracy_avgs[alg] = test_accuracy_avgs[alg] / iterations_count
    test_time_avgs[alg] = test_time_avgs[alg] / iterations_count
    test_accuracy_table[alg].append(test_accuracy_avgs[alg])
    test_accuracy_table[alg].append(test_results[alg]['worst_accuracy']['accuracy'])
    test_accuracy_table[alg].append(test_results[alg]['best_accuracy']['accuracy'])
    test_time_table[alg].append(test_time_avgs[alg])
    test_time_table[alg].append(test_results[alg]['worst_time']['elapsed_time'])
    test_time_table[alg].append(test_results[alg]['best_time']['elapsed_time'])

test_accuracy_table = pd.DataFrame(test_accuracy_table)
test_time_table = pd.DataFrame(test_time_table)

print()
show_title("Medias de acuracias no teste")
print(test_accuracy_table)
#mostrar_dataframe_interativo(test_accuracy_table,min(len(test_accuracy_table),iterations_count)+1,min(len(test_accuracy_table),iterations_count)+1)
print()
show_title("Medias de tempos no teste")
print(test_time_table)
#mostrar_dataframe_interativo(test_time_table,min(len(test_time_table),iterations_count)+1,min(len(test_time_table),iterations_count)+1)

   repetição      knn       dt      svm      mlp
0          1 0.871028 0.899065 0.852336 0.854206
1          2 0.871028 0.899065 0.852336 0.846729
2          3 0.871028 0.899065 0.852336 0.820561
3          4 0.871028 0.899065 0.852336 0.816822
4          5 0.871028 0.899065 0.852336 0.820561
5          6 0.871028 0.899065 0.852336 0.809346
6          7 0.871028 0.899065 0.852336 0.852336
7          8 0.871028 0.899065 0.852336 0.848598
8          9 0.871028 0.899065 0.852336 0.770093
9         10 0.871028 0.899065 0.852336 0.811215
10       AVG 0.871028 0.899065 0.852336 0.825047
11     WORST 0.871028 0.899065 0.852336 0.770093
12      BEST 0.871028 0.899065 0.852336 0.854206



   repetição      knn       dt      svm      mlp
0          1 0.061419 0.008631 0.883536 0.063105
1          2 0.050206 0.014620 0.872864 0.042315
2          3 0.056944 0.015626 0.878054 0.047962
3          4 0.050031 0.013637 0.865670 0.056935
4          5 0.054760 0.015804 0.878372 0.057082
5          6 0.049103 0.016704 0.912283 0.057376
6          7 0.051444 0.017745 0.900905 0.051225
7          8 0.053576 0.015630 0.866369 0.083664
8          9 0.065006 0.016684 0.858203 0.100377
9         10 0.052346 0.015626 0.911982 0.044619
10       AVG 0.054483 0.015071 0.882824 0.060466
11     WORST 0.065006 0.017745 0.912283 0.100377
12      BEST 0.049103 0.008631 0.858203 0.042315


In [21]:
#Aplicando estatistica de Kruskal-Wallis
df_statics = validation_accuracy_table.iloc[:-3] #elimina as 3 ultimas linhas, que contem as medias, melhores e piores

kruskal_result = kruskal(df_statics['knn'], df_statics['dt'], df_statics['svm'], df_statics['mlp'])
show_title("Resultado do teste de Kruskal-Wallis:")
print("Estatística:", kruskal_result.statistic)
print("p-valor:", kruskal_result.pvalue)

alpha = 0.05
if kruskal_result.pvalue < alpha:
    print('Isso indica que os valores das acuracias apuradas são significativamente diferentes.')
else:
    print('Isso indica que os valores das acuracias apuradas não são significativamente diferentes.')

Estatística: 23.182278843401082
p-valor: 3.6999810265052e-05
Isso indica que os valores das acuracias apuradas são significativamente diferentes.


In [22]:
# # Comparação par-a-par com Mann-Whitney

show_title("\nTeste de Mann-Whitney (comparação dois a dois):")
classifiers = {
    'MLP': df_statics['mlp'],
    'DT': df_statics['dt'],
    'SVM': df_statics['svm'],
    'KNN': df_statics['knn']
}
alpha = 0.05

for (name1, acc1), (name2, acc2) in itertools.combinations(classifiers.items(), 2):
    mannwhitney_result = mannwhitneyu(acc1, acc2, alternative='two-sided')
    print(f"Comparação {name1} vs {name2}:")
    print("  Estatística:", mannwhitney_result.statistic)
    print("  p-valor:", mannwhitney_result.pvalue)
    if mannwhitney_result.pvalue < alpha:
        print('  Isso indica que os valores das acuracias apuradas são significativamente diferentes.')
    else:
        print('  Isso indica que os valores das acuracias apuradas não são significativamente diferentes.')



Comparação MLP vs DT:
  Estatística: 0.0
  p-valor: 7.521176571182515e-05
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
Comparação MLP vs SVM:
  Estatística: 4.0
  p-valor: 0.00037396107782172684
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
Comparação MLP vs KNN:
  Estatística: 0.0
  p-valor: 0.00010582785755376433
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
Comparação DT vs SVM:
  Estatística: 58.0
  p-valor: 0.5619440752989311
  Isso indica que os valores das acuracias apuradas não são significativamente diferentes.
Comparação DT vs KNN:
  Estatística: 44.5
  p-valor: 0.6950038567413858
  Isso indica que os valores das acuracias apuradas não são significativamente diferentes.
Comparação SVM vs KNN:
  Estatística: 22.0
  p-valor: 0.03706158629159534
  Isso indica que os valores das acuracias apuradas são significativamente diferentes.
